# step by step notebook for reproducing the workflow from RT pred manuscript 

Ensure that the requriements are met i.e.
- rdkit 
- pytorch 
- deepchem
- chemprop
- etc.. 




In [1]:
import os 
from functions.featurizers import get_METLIN_data, get_features 
from functions.data_splitting import data_splitter, feature_splitter_csv, feature_splitter_diskdatasets


Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading some PyTorch models, missing a dependency. No module named 'torch'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch'
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'torch'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


## Featurization and data splitting

This workflow is run using the METLIN SMRT Dataset published by Domingo-Almenara et al. https://www.nature.com/articles/s41467-019-13680-7 

But any dataset could be used, however, it must contain columns 'rt' and 'smiles'

By default we remove non-retained compounds from the METLIN SMRT dataset and subsample (2000, randomly)

In [2]:
path_to_data = get_METLIN_data(sampling = 2000, only_retained = True)

saved to:  ../data/metlin_smrt\sample_dataset_2000_datapoints.csv
             id                                             smiles      rt
37952  53040624             CCc1cc(C(=O)NC2CC2)nc2cc(-c3cccs3)nn12  1009.8
26748  24180719  CCCS(=O)(=O)Nc1ccc(F)c(C(=O)c2c[nH]c3ncc(Cl)cc...   877.8
59423  53180324  COc1cccc(N2C(=O)N(Cc3cccc(F)c3)S(=O)(=O)c3cccn...  1007.3


There are a number of features that we use for modeling. 

- LogD descriptor set (pH 0.5 to 7.4)
- ECFP4-2048 fingerprints
- RDKit descriptors 
- molecular graph convolutions 

based on the feature_list ``` feature_list = ['logD', 'ecfp4', 'rdkit', 'molgraphconv]``` it is possible to define which features to be calculated. 

Note that the ECFP4 and RDKit descriptors come in two variants: a flat .csv file or a DeepChem DiskData object. This choice will depend on downstream application and models.


OBS! LogD calculations are based on the proprietary software cxcalc from Chemaxon (https://docs.chemaxon.com/display/docs/cxcalc-command-line-tool.md). A path to a chemaxon licence is needed for this calculation, as well as a specified path to the chemaxon tool. If not provided, the LogD calculations are skipped


In [4]:
path_to_features = '../data/metlin_smrt/features/' 
if not os.path.exists(path_to_features):
    os.makedirs(path_to_features)

feature_list = ['logD', 
                'ecfp4', 
                'rdkit', 
                'molgraphconv'
                ]

get_features(path_to_data, 
             path_to_features, feature_list, 
             path_to_chemaxon_licence= r"G:\Nuevolution\ChemAxon\licenses\license.cxl", 
             path_to_chemaxon_tools = r"C:\nuevolution\knime_temp")

ChemAxon-based LogD calculations - to .CSV
*** NA values in: ../data/metlin_smrt/features/logd_calculations


### Data Splitting 

The data is split into train/test. In addition the train dataset is split 5 times to allow for 5-fold cross validation. 

In [ ]:
path_to_splits = '../data/metlin_smrt/data_splits/'
if not os.path.exists(path_to_splits):
    os.makedirs(path_to_splits)

data_splitter(path_to_data, path_to_splits, number_of_splits = 5)

The following functions will take the features already calculated and bundle them according to the above splits

In [ ]:

#* splitting the .CSV-based features according to the data splits
feature_splitter_csv(path_to_features, path_to_splits, save_folder_name = 'cv_splits')


#* splitting the diskDataset-based features according to the data splits
feature_splitter_diskdatasets(path_to_features,path_to_splits, save_folder_name = 'cv_splits')